In [18]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from keras.applications import VGG16
from keras.models import Model, Sequential
from keras.layers import Flatten, Dense, Input, BatchNormalization
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils
from keras.datasets import cifar10
# import matplotlib
# matplotlib.use('Agg')

Test the hypothesis of pretrained model on imagenet against adversarial perturbations

In [5]:
model = VGG16(include_top=False, weights='imagenet', input_tensor=Input(shape=(32, 32, 3)))

58859520/58889256 [============================>.] - ETA: 0s  

In [6]:
# check the model output dimension
model.output

<tf.Tensor 'MaxPool_4:0' shape=(?, 1, 1, 512) dtype=float32>

In [7]:
# check weather we can take the conv feature map
batch_size = 128
nb_classes= 10
nb_epoch = 100 
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
trY = np_utils.to_categorical(y_train)
teY = np_utils.to_categorical(y_test)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.
X_test /= 255.

170450944/170498071 [============================>.] - ETA: 0s   

In [8]:
# conv feature maps
trX = model.predict(X_train)
teX = model.predict(X_test)

In [12]:
print trX.shape
print teX.shape
model.output

(50000, 1, 1, 512)
(10000, 1, 1, 512)


<tf.Tensor 'MaxPool_4:0' shape=(?, 1, 1, 512) dtype=float32>

In [30]:
# construct thte top layer
model2 = Sequential([
    BatchNormalization(axis=-1, input_shape=trX.shape[1:]),
    Flatten(),
    Dense(4096, activation='relu'),
    BatchNormalization(axis=-1),
    Dense(4096, activation='relu'),
    BatchNormalization(axis=-1),
    Dense(10, activation='softmax')
])

model2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_15 (Flatten)             (None, 512)           0           flatten_input_3[0][0]            
____________________________________________________________________________________________________
dense_37 (Dense)                 (None, 4096)          2101248     flatten_15[0][0]                 
____________________________________________________________________________________________________
dense_38 (Dense)                 (None, 4096)          16781312    dense_37[0][0]                   
____________________________________________________________________________________________________
dense_39 (Dense)                 (None, 10)            40970       dense_38[0][0]                   
Total params: 18,923,530
Trainable params: 18,923,530
Non-trainable params: 0
_____________

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-10, mode='auto', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=25, verbose=1)

In [ ]:
model2.compile(optimizer='adamax', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(trX, trY, batch_size=batch_size, nb_epoch=nb_epoch, validation_data=(teX, teY), shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/200


In [ ]:
model2.evaluate(teX, teY)
model2.save('vgg16_model_tained_top.h5')
model2.save_weights('vgg16_weights_trained_top.h5')